# **The Modeling and Control Of Manipulators**
## **Goals for today**

---



During today practice we will:
*   Revise the Dynamics of Mechanical Systems.
*   Implement the simple multidof PD controller. 
*   Study the gravity compensation and inverse dynamics (fb)


## **Dynamics of Manipulators**

Equation of motion for most mechanical systems may be written in following form:
\begin{equation}
\mathbf{Q} - \mathbf{Q}_d = \mathbf{D}(\mathbf{q})\ddot{\mathbf{q}} + \mathbf{C}(\mathbf{q},\dot{\mathbf{q}})\dot{\mathbf{q}} + \mathbf{g}(\mathbf{q})  = \mathbf{D}(\mathbf{q})\ddot{\mathbf{q}} + \mathbf{c}(\mathbf{q},\dot{\mathbf{q}}) + \mathbf{g}(\mathbf{q}) = \mathbf{D}(\mathbf{q})\ddot{\mathbf{q}} + \mathbf{h}(\mathbf{q},\dot{\mathbf{q}})
\end{equation}

where:
* $\mathbf{Q} \in \mathbb{R}^n $ - generalized forces corresponding to generilized coordinates
* $\mathbf{Q}_d \in \mathbb{R}^n $ - generalized disippative forces (for instance friction)
* $\mathbf{q} \in \mathbb{R}^{n}$ - vector of generilized coordinates
* $\mathbf{D} \in \mathbb{R}^{n \times n}$ - positive definite symmetric inertia matrix 
* $\mathbf{C} \in \mathbb{R}^{n \times n}$ - describe 'coefficients' of centrifugal and Coriolis forces
* $\mathbf{g} \in \mathbb{R}^{n}$ - describes effect of gravity and other position depending forces
* $\mathbf{h} \in \mathbb{R}^n $ - combined effect of $\mathbf{g}$ and $\mathbf{C}$






#### **Lagrange-Euler Equations**

In order to find the EoM we will use the Lagrange-Euler equations:
\begin{equation}
{\frac  {d}{dt}}\left({\frac  {\partial \mathcal{L}}{\partial {\dot{\mathbf{q}}}_{i}}}\right)-{\frac  {\partial \mathcal{L}}{\partial \mathbf{q}_{i}}}=\mathbf{Q}_i -\frac{\partial \mathcal{R}}{\partial \mathbf{\dot{q}}_i},\quad i = 1,2,\dots, n
\end{equation}

where:
* $\mathcal{L}(\mathbf{q},\dot{\mathbf{q}}) \triangleq E_K - E_\Pi \in \mathbb{R}$ Lagrangian of the system  
* $\mathcal{R} \in \mathbb{R}$ Rayleigh function  (describes energy dissipation)



### **Example: Remotely Driven Two Link Manipulator**

---


For instance let's consider remotely driven rigid 2-DoF manipulator:
<p><img alt="Remote manipulator" height="250" src="https://drive.google.com/uc?id=1nY6dY3jmwtfXuIy0p1ZUXs3TNC0iSqf5" align="center" hspace="50px" vspace="0px"></p>



#### **Energy**
Kinetic and potential energy of this system is equal to:
\begin{equation}
E_K = \frac{1}{2}\sum_{j=1}^{2}  m_j\dot{\mathbf{r}}_j^T\dot{\mathbf{r}}_j  +  J_j \dot{\alpha}^2_i, \quad E_\Pi = \sum_{j=1}^{n}  m_i g y_i  
\end{equation}


Where $\mathbf{r}_j$ is Cartesian coordinates of mass $m_j$ calculated via forward kinematics as:
\begin{equation}
\mathbf{r}_1
=
\begin{bmatrix}
x_1 \\
y_1
\end{bmatrix}
=
\begin{bmatrix}
l_1 \cos \alpha_1 \\
l_1 \sin \alpha_1
\end{bmatrix}, \quad
\mathbf{r}_2
=
\begin{bmatrix}
x_2 \\
y_2
\end{bmatrix}
=
\begin{bmatrix}
l_1 \cos \alpha_1 + l_2 \cos \alpha_2 \\
l_1 \sin \alpha_1 + l_2 \sin \alpha_2
\end{bmatrix}
\end{equation}




Apart from kinetic and potential energy there are two dissipative elements in this system, namely "dampers" with coefficients $b_1, b_2$ (viscous friction), their dissipation function is given as:
\begin{equation}
\mathcal{R} = \frac{1}{2}\sum_{j=1}^{n}  b_j \dot{\alpha}^2_j
\end{equation}


Defining generalized coordinates as $\mathbf{q} = [\alpha_1,\alpha_2]^T$ and forces as $\mathbf{Q} = [u_1,u_2]^T$ one can apply Lagrange formalism to obtain equations of motion as:

\begin{equation}
\begin{cases} 
I_1\ddot{\alpha}_1 + l_1^2 (m_1 + m_2) \ddot{\alpha}_1 + l_1 l_2 m_2 \cos(\alpha_1 - \alpha_2)\ddot{\alpha}_2 + 
l_1 l_2 m_2 \sin(\alpha_1 - \alpha_2)\dot{\alpha}^2_2
+l_1 m_1 g \cos \alpha_1 + l_1 m_2 g \cos \alpha_2 + b_1 \dot{\alpha}_1 =u_1 \\
l_1 l_2 m_2 \cos(\alpha_1 - \alpha_2)\ddot{\alpha}_1 + I_2 \ddot{\alpha} + l_2^2 m_2 \ddot{\alpha}_2 - l_2 m_2 l_1 \sin(\alpha_1 - \alpha_2)\dot{\alpha}^2_1 + l_2 m_2 g \cos \alpha_2+ b_2 \dot{\alpha}_2 = u_2
\end{cases}
\end{equation}




Now we can find the $\mathbf{D}, \mathbf{C}, \mathbf{g}$ to be:
\begin{equation}
\mathbf{D} = 
\begin{bmatrix}
l_1^2 (m_1 + m_2)+ J_1 & l_1 l_2 m_2  \cos(\alpha_1 - \alpha_2)\\
l_1 l_2 m_2 \cos(\alpha_1 - \alpha_2) & l_2^2 m_2+ J_2
\end{bmatrix}, \quad
\mathbf{Q}_d = 
\begin{bmatrix}
b_1 \dot{\alpha}_1 \\
b_2 \dot{\alpha}_2
\end{bmatrix}
\end{equation}

\begin{equation}
\mathbf{c} = \begin{bmatrix}
l_1 l_2 m_2 \sin(\alpha_1 - \alpha_2)\dot{\alpha}^2_2 \\
- l_2 m_2 l_1 \sin(\alpha_1 - \alpha_2)\dot{\alpha}^2_1
\end{bmatrix}
, \quad
\mathbf{g} = \begin{bmatrix}
l_1 m_1 g \cos \alpha_1 + l_1 m_2 g \cos \alpha_1 \\
 l_2 m_2 g \cos \alpha_2
\end{bmatrix}
\end{equation}

### **Dynamics in State Space Form**
Choosing state as $\mathbf{x} = [\mathbf{x}_1,\mathbf{x}_2]^T= [\mathbf{q},\dot{\mathbf{q}}]^T$ and control as $\mathbf{Q} = [u_1,u_2]^T$ we can easily rewrite equations above as:
\begin{equation}
\dot{\mathbf{x}}
=
\begin{bmatrix}
\dot{\mathbf{x}}_1 \\ 
\dot{\mathbf{x}}_2
\end{bmatrix}
=
\begin{bmatrix}
\dot{\mathbf{q}} \\ 
\ddot{\mathbf{q}}
\end{bmatrix}
=
\begin{bmatrix}
\mathbf{x}_2 \\
\mathbf{D}(\mathbf{x}_1)^{-1} \big(\mathbf{u} - \mathbf{Q}_d -  \mathbf{h}(\mathbf{x}_1, \mathbf{x}_2) \big)
\end{bmatrix}
\end{equation}

Once system is written in state space form - one can use conventional ODE solvers to simulate it.

### **Tasks:**

1. Implement the dynamics of two link manipulator and model the free response
2. Propose the PD regulator to regulate the state to different desired values
3. Introduce the gravity compensation 
4. Change desired trajectory to be the harmonical signal with different frequancies and amplitudes, discuss the response
5. Implement the inverse dynamics controller. How did the response change?
6. Vary the load parameter for simulation while keeping the same value for controller, discuss the change in system response both for regulation and tracking.



### **System Parameters:**

In [ ]:
# Manipulator parameters
l = 0.3, 0.3
m = 0.5, 3.0
J = 0.01, 0.01
b = 0.02, 0.02
g = 9.81
params = l, m, J, b, g